# Deep Temporal Transformer for Financial Fraud Detection
**Colab Pro GPU-Optimized Notebook**

This notebook runs the complete fraud detection pipeline on GPU with optimized settings for Colab Pro.

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install dependencies for Colab Pro
!pip install -q torch torchvision torchaudio
!pip install -q scikit-learn pandas matplotlib seaborn numpy
print('✅ Dependencies installed')

In [ ]:
# Upload and extract project files (since we can't clone from GitHub in this demo)
# Alternative: Upload the project as a zip file to Colab
import os
import sys

# For demo purposes, we'll create the package structure directly
!mkdir -p deep_temporal_transformer/{configs,data,models,training,evaluation,utils,examples}
print('📁 Project structure created')

# Add to Python path
sys.path.append('/content')
print('🐍 Python path updated')

In [ ]:
# Create minimal working version for Colab
%%writefile deep_temporal_transformer/__init__.py
"""Deep Temporal Transformer Package"""
__version__ = "1.0.0"

In [ ]:
# Quick demo with synthetic data (Colab-optimized)
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")

# Generate synthetic fraud data
np.random.seed(42)
torch.manual_seed(42)

n_samples = 10000  # Smaller for Colab
n_features = 10
fraud_rate = 0.02

# Create synthetic transaction data
X = np.random.randn(n_samples, n_features)
y = np.random.binomial(1, fraud_rate, n_samples)

# Add fraud patterns
fraud_indices = np.where(y == 1)[0]
X[fraud_indices, :3] += np.random.normal(2, 0.5, (len(fraud_indices), 3))

print(f"📊 Generated {n_samples} samples with {y.sum()} fraud cases ({y.mean():.1%} fraud rate)")

In [ ]:
# Simple transformer model for Colab
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=128, 
            batch_first=True, activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        # Add sequence dimension for transformer
        if x.dim() == 2:
            x = x.unsqueeze(1)  # (batch, 1, features)
        
        h = self.input_proj(x)
        h = self.transformer(h)
        h = h.mean(dim=1)  # Global average pooling
        return self.classifier(h).squeeze(-1)

print('🤖 Simple Transformer model defined')

In [ ]:
# Train and evaluate models
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"📈 Training set: {X_train.shape}, Test set: {X_test.shape}")

# Baseline: Random Forest
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
rf_prob = rf.predict_proba(X_test_scaled)[:, 1]

rf_f1 = f1_score(y_test, rf_pred)
rf_auc = roc_auc_score(y_test, rf_prob)

print(f"🌲 Random Forest - F1: {rf_f1:.4f}, AUC: {rf_auc:.4f}")

In [ ]:
# Train Simple Transformer
model = SimpleTransformer(input_dim=n_features).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(20.0).to(device))  # Handle imbalance

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)

# Training loop
model.train()
batch_size = 256
epochs = 20

for epoch in range(epochs):
    total_loss = 0
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(range(0, len(X_train_tensor), batch_size)):.4f}")

print('🎯 Transformer training completed')

In [ ]:
# Evaluate Transformer
model.eval()
with torch.no_grad():
    logits = model(X_test_tensor)
    probs = torch.sigmoid(logits).cpu().numpy()
    preds = (probs > 0.5).astype(int)

transformer_f1 = f1_score(y_test, preds)
transformer_auc = roc_auc_score(y_test, probs)

print(f"🤖 Simple Transformer - F1: {transformer_f1:.4f}, AUC: {transformer_auc:.4f}")

# Results comparison
print("\n" + "="*50)
print("📊 FRAUD DETECTION RESULTS")
print("="*50)
print(f"Random Forest:     F1={rf_f1:.4f}, AUC={rf_auc:.4f}")
print(f"Simple Transformer: F1={transformer_f1:.4f}, AUC={transformer_auc:.4f}")
print("\n✅ Colab Pro Demo Completed Successfully!")

In [ ]:
# Optional: Visualize results
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, confusion_matrix
import seaborn as sns

# ROC Curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_prob)
fpr_tr, tpr_tr, _ = roc_curve(y_test, probs)

plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC={rf_auc:.3f})', linewidth=2)
plt.plot(fpr_tr, tpr_tr, label=f'Transformer (AUC={transformer_auc:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.grid(True, alpha=0.3)

# Confusion Matrix
plt.subplot(1, 2, 2)
cm = confusion_matrix(y_test, preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Fraud'], 
            yticklabels=['Normal', 'Fraud'])
plt.title('Transformer Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.tight_layout()
plt.show()

print("📈 Visualization completed")